# Dependencies installation and import 


In [ ]:
!pip install unidecode
!pip install pandas
!pip install re
!pip install pickle
!pip install sklearn
!pip install numpy
!pip install matplotlib
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
import os
import re
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import nltk
import unidecode
from string import ascii_letters, punctuation
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Data processing



In [ ]:
wikihow = pd.read_csv(r'/content/drive/MyDrive/summarization/wikihowAll.csv')

In [ ]:
wikihow = wikihow.astype(str)
wikihow=wikihow[wikihow['text'].isnull()==False]
wikihow=wikihow[wikihow['headline'].isnull()==False]
wikihow=wikihow[wikihow['text']!='nan']
wikihow.drop_duplicates(subset=['text'],inplace=True)
wikihow['word_count'] = wikihow['text'].str.count(' ') + 1
wikihow['headline_count'] = wikihow['headline'].str.count(' ') + 1

In [ ]:
path = '/content/drive/MyDrive/summarization/wikihow_all.pkl'

In [ ]:
# pd.to_pickle(wikihow, path)

In [ ]:
# wikihow_ = pd.read_pickle(path)

In [ ]:
contraction = { 
    "ain't": "is not", 
    "aren't": "are not", 
    "can't": "cannot", 
    "'cause": "because", 
    "could've": "could have", 
    "couldn't": "could not",
    "didn't": "did not", 
    "doesn't": "does not", 
    "don't": "do not", 
    "hadn't": "had not", 
    "hasn't": "has not", 
    "haven't": "have not",
    "haven t": "have not",
    "he'd": "he would", 
    "he'll": "he will", 
    "he's": "he is", 
    "how'd": "how did", 
    "how'd'y": "how do you", 
    "how'll": "how will", 
    "how's": "how is",
    "I'd": "I would", 
    "I'd've": "I would have", 
    "I'll": "I will", 
    "I'll've": "I will have", 
    "I'm": "I am", 
    "I've": "I have", 
    "i'd": "i would",
    "i'd've": "i would have", 
    "i'll": "i will", 
    "i'll've": "i will have", 
    "i'm": "i am", 
    "i've": "i have", 
    "isn't": "is not", 
    "it'd": "it would",
    "it'd've": "it would have", 
    "it'll": "it will", 
    "it'll've": "it will have", 
    "it's": "it is", 
    "let's": "let us", 
    "ma'am": "madam",
    "mayn't": "may not", 
    "might've": "might have", 
    "mightn't": "might not", 
    "mightn't've": "might not have", 
    "must've": "must have",
    "mustn't": "must not", 
    "mustn't've": "must not have", 
    "needn't": "need not", 
    "needn't've": "need not have", 
    "o'clock": "of the clock",
    "oughtn't": "ought not", 
    "oughtn't've": "ought not have", 
    "shan't": "shall not", 
    "sha'n't": "shall not", 
    "shan't've": "shall not have",
    "she'd": "she would", 
    "she'd've": "she would have", 
    "she'll": "she will", 
    "she'll've": "she will have", 
    "she's": "she is",
    "should've": "should have", 
    "shouldn't": "should not", 
    "shouldn't've": "should not have", 
    "so've": "so have", 
    "so's": "so as",
    "this's": "this is", 
    "that'd": "that would", 
    "that'd've": "that would have", 
    "that's": "that is", 
    "there'd": "there would",
    "there'd've": "there would have", 
    "there's": "there is", 
    "here's": "here is", 
    "they'd": "they would", 
    "they'd've": "they would have",
    "they'll": "they will", 
    "they'll've": "they will have", 
    "they're": "they are", 
    "they've": "they have", 
    "to've": "to have",
    "wasn't": "was not", 
    "we'd": "we would", 
    "we'd've": "we would have", 
    "we'll": "we will", 
    "we'll've": "we will have", 
    "we're": "we are",
    "we've": "we have", 
    "weren't": "were not", 
    "what'll": "what will", 
    "what'll've": "what will have", 
    "what're": "what are",
    "what's": "what is", 
    "what've": "what have", 
    "when's": "when is", 
    "when've": "when have", 
    "where'd": "where did", 
    "where's": "where is",
    "where've": "where have", 
    "who'll": "who will", 
    "who'll've": "who will have", 
    "who's": "who is", 
    "who've": "who have",
    "why's": "why is", 
    "why've": "why have", 
    "will've": "will have", 
    "won't": "will not", 
    "won't've": "will not have",
    "would've": "would have", 
    "wouldn't": "would not", 
    "wouldn't've": "would not have", 
    "y'all": "you all",
    "y'all'd": "you all would", 
    "y'all'd've": "you all would have", 
    "y'all're": "you all are", 
    "y'all've": "you all have",
    "you'd": "you would", 
    "you'd've": "you would have", 
    "you'll": "you will", 
    "you'll've": "you will have",
    "you're": "you are", 
    "you've": "you have"}

In [ ]:
def clean(text):
  
    text = text.lower()
    text = unidecode.unidecode(text)
    text = text.replace('&', ' and ')
    text = text.replace('@', ' at ')
    text = ' '.join([contraction[t] if t in contraction else t for t in text.split(" ")])
    
    punct = '|'.join('\\' + char for char in punctuation)
    punct = r'['+punct+']'
    text = re.sub(punct, ' ', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"'s\b","",text)
    text = re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", text)

    text = text.strip(" ")
    text = re.sub('"','', text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(' +',' ', text).strip() 

    if len(text) != 0 and text[len(text)-1].isdigit():
        text = text[0:len(text)-1]
    return text



In [ ]:
wikihow["clean_title"] = wikihow['title'].map(lambda text: clean(text))

In [ ]:
wikihow["clean_text"] = wikihow['text'].map(lambda text: clean(text))

In [ ]:
wikihow["clean_headline"] = wikihow['headline'].map(lambda text: clean(text))

## Load WikiHow (test)


In [ ]:
path = '/content/drive/MyDrive/summarization/wikihow_all_clean_nn.pkl'

In [ ]:
# pd.to_pickle(wikihow, path)

In [ ]:
# wikihow = pd.read_pickle(path)

# Shorter articles

In [ ]:
wikihow['headline_count'] = wikihow['headline'].str.count(' ') + 1

In [ ]:
shorty = wikihow["word_count"] < MAX_INPUT_LENGTH
shorter_articles = wikihow[shorty].copy()

shortyh = shorter_articles["headline_count"] < MAX_OUTPUT_LENGTH
shorter_articles = shorter_articles[shortyh].copy()

## Prepare data

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(shorter_articles['clean_text'],shorter_articles['clean_headline'],test_size=0.25,random_state=0,shuffle=True) 

In [4]:
path1 = '/content/drive/MyDrive/summarization/xtrain.pkl'
path2 = '/content/drive/MyDrive/summarization/ytrain.pkl'
path3 = '/content/drive/MyDrive/summarization/xtest.pkl'
path4 = '/content/drive/MyDrive/summarization/ytest.pkl'


In [5]:
import pickle
def pick(path1, data):
  with open(path1, 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [6]:
def unpick(path1):
  with open(path1, 'rb') as f:
    data = pickle.load(f)
    return data

In [ ]:
# pick(path1, x_train)
# pick(path2, y_train)
# pick(path3, x_test)
# pick(path4, y_test)

In [7]:
# x_train = unpick(path1)
# y_train = unpick(path2)
# x_test = unpick(path3)
# y_test = unpick(path4)

In [ ]:
START = 0
END = 1

In [ ]:
class Text:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: "START", 1: "END"}
    self.n_words = 2  

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [ ]:
def readText(x_train, y_train):
  textt=np.array(x_train)
  summary=np.array(y_train)
  pairs = [[textt[i],summary[i]] for i in range(len(textt))]
  text = Text(textt)
  reference_summary = Text(summary)
  for pair in pairs:
    text.addSentence(pair[0])
    reference_summary.addSentence(pair[1])

  return text, reference_summary, pairs

In [ ]:
input_text, output_text, pairs = readText(x_train, y_train )

# LSTM with Attention

In [ ]:
INPUT_SHORTER_LENGTH = 300

MAX_INPUT_LENGTH = INPUT_SHORTER_LENGTH
MAX_OUTPUT_LENGTH = 150

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(EncoderLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.input_size = input_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.LSTM = nn.LSTM(hidden_size, hidden_size)

  def forward(self, input, hidden):
    embedded = self.embedding(input).view(1, 1, -1)
    # TODO
    output, hidden = self.LSTM(embedded, hidden)
    return output, hidden

  def initHidden(self):
    return (torch.zeros(1, 1, self.hidden_size, device=device),torch.zeros(1, 1, self.hidden_size, device=device)) 

In [ ]:
class AttentionLSTM(nn.Module):
  def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_INPUT_LENGTH):
    super(AttentionLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout_p = dropout_p
    self.max_length = max_length

    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.attention = nn.Linear(self.hidden_size*2 , self.max_length)
    self.combine_attention = nn.Linear(self.hidden_size*2 , self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)
    self.LSTM = nn.LSTM(self.hidden_size, self.hidden_size)
    self.out = nn.Linear(self.hidden_size, self.output_size)


  def forward(self, input, hidden, encoder_outputs):

    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    attention_weights = F.softmax(self.attention(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
    applied_attention = torch.bmm(attention_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

    concatenation = torch.cat((embedded[0], applied_attention[0]), 1)
    attention_result = self.combine_attention(concatenation).unsqueeze(0)

    relu = F.relu(attention_result)
    output, hidden = self.LSTM(relu, hidden)

    output = F.log_softmax(self.out(output[0]), dim=1)
    return output, hidden, attention_weights

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 300


# Train

In [ ]:
encoder = EncoderLSTM(input_text.n_words, hidden_size).to(device)
decoder = AttentionLSTM(hidden_size, output_text.n_words, dropout_p=0.1).to(device)

In [ ]:
def train_model(encoder, decoder,input_text, output_text, n_iters, debug_every=1000, learning_rate=0.01, input_length_treshold = INPUT_SHORTER_LENGTH, path = '/content/drive/MyDrive/summarization/'):

  debug_loss_total = 0  
  plot_losses = []  

  input_output_pairs = [tensorsFromPair(random.choice(pairs), input_text, output_text)
                      for i in range(n_iters)]
  encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
  criterion = nn.NLLLoss()
  
  for iter in range(1, n_iters + 1):
      
    training_pair = input_output_pairs[iter - 1]
    input_tensor = training_pair[0]
    target_tensor = training_pair[1]

    input_length = input_tensor.size(0)
    if(input_length > input_length_treshold):
      print('err')
      print(input_length)
      continue
      
    loss = train(input_tensor, target_tensor, encoder,decoder, encoder_optimizer, decoder_optimizer, criterion )
    debug_loss_total += loss
    
    if iter % debug_every == 0:
      debug_loss_avg = debug_loss_total / debug_every
      debug_loss_total = 0
      plot_losses.append(debug_loss_avg)
      torch.save(encoder.state_dict(), path + 'encoder_new3.dict')
      torch.save(decoder.state_dict(), path + 'decoder_new3.dict')
      print(' (%d %d%%) %.4f' % (iter, iter / n_iters * 100, debug_loss_avg))

  showPlot(plot_losses)

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, learning_rate=0.01,  max_length=MAX_INPUT_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for i in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
        encoder_outputs[i] = encoder_output[0, 0]

    decoder_input = torch.tensor([[START]], device=device)

    decoder_hidden = encoder_hidden

    for i in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
        decoder_input, decoder_hidden, encoder_outputs) 
      
        loss += criterion(decoder_output, target_tensor[i])
        decoder_input = target_tensor[i]  # Teacher forcing


    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def tensorsFromPair(pair, input_text, output_text):
    input_indexes = [input_text.word2index[word] for word in pair[0].split(' ')]
    input_indexes.append(END)
    input_tensor = torch.tensor(input_indexes, dtype=torch.long, device=device).view(-1, 1)

    target_indexes = [output_text.word2index[word] for word in pair[1].split(' ')]
    target_indexes.append(END)
    target_tensor = torch.tensor(target_indexes, dtype=torch.long, device=device).view(-1, 1)
    return (input_tensor, target_tensor)

In [ ]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
    plt.plot(points)

In [ ]:
train_model(encoder, decoder, input_text, output_text, 122000, debug_every=100)

# Evaluation

In [ ]:
def evaluate(encoder, decoder, pairs, input_test, output_test, debug_every=1000, path = '/content/drive/MyDrive/summarization/'):
  text=[]
  headline=[]
  predicted_headline=[] 
  path1 =  path + 'text_lstm.pkl'
  path2 =  path + 'headline_lstm.pkl'
  path3 =  path + 'summary_lstm.pkl'

  n = len(pairs)
  print('Producing summaries: ')
  count = 0
  for i in range(0, n):
    pair = pairs[i]
        
    if len(pair[0].split()) >= MAX_INPUT_LENGTH :
      print(i)
      print('err')
      continue
    else:
      if i%debug_every == 0 :
        print(i*100/n,"% complete")
        print(i)
        pick(path1, text)
        pick(path2, headline)
        pick(path3, predicted_headline)

      text.append(pair[0])
      headline.append(pair[1])
      predicted_words, attentions = produceSummary(encoder, decoder, input_test, output_test, pair[0])
      predicted_sentence = ' '.join(predicted_words)
      predicted_headline.append(predicted_sentence)

      count += 1

  return (text,headline,predicted_headline)

In [ ]:
def produceSummary(encoder, decoder, input_text, summary_text, sentence, max_length=MAX_INPUT_LENGTH):
    with torch.no_grad():
        indexes = [input_text.word2index[word] for word in sentence.split(' ')]
        indexes.append(END)
        input_tensor = torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)
        input_length = input_tensor.size()[0]

        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for e in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[e],encoder_hidden)
            encoder_outputs[e] += encoder_output[0, 0]

        decoder_input = torch.tensor([[START]], device=device)  

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for d in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[d] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == END:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(summary_text.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:d + 1]

In [ ]:
input_test, output_test, pairs_test = readText(x_test, y_test )


In [ ]:
path =  '/content/drive/MyDrive/summarization/summaries.pkl'
summaries=pd.DataFrame()

summaries['clean_text'] = ""
summaries['clean_headline'] = ""
summaries['summary'] = ""

text, headline, predicted_headline = evaluate(encoder, decoder, pairs_test, input_test, output_test)



summaries['clean_text']=text
summaries['clean_headline']=headline
summaries['summary']=predicted_headline

In [ ]:
path =  '/content/drive/MyDrive/summarization/summaries.pkl'

In [ ]:
pick(path, summaries)

## Installation von nlg-eval, functions

In [ ]:
!pip3 install git+https://github.com/Maluuba/nlg-eval.git@master


In [ ]:
!nlg-eval --setup

In [ ]:
import io

def write_list_to(filename, da_list):
  with open(filename, 'w') as f:
    for item in da_list:
      f.write("%s\n" % item)


def calculate_bleurt(filename):
  with io.open(filename,'r') as f:
    bl = f.readlines()

  b = [float(i) for i in bl]
  score = sum(b)/len(b)
  print('score for ' + filename)
  print(score)


In [ ]:
to_eval_hyp = summaries['summary']
to_eval_ref = summaries['clean_headline']

In [ ]:
h_filename = '/content/drive/MyDrive/summarization/evaluation/lstm_hypothesis.txt'
r_filename = '/content/drive/MyDrive/summarization/evaluation/lstm_reference.txt'

In [ ]:
write_list_to(h_filename, to_eval_hyp)
write_list_to(r_filename, to_eval_ref)

In [ ]:
  !nlg-eval --hypothesis=/content/drive/MyDrive/summarization/evaluation/lstm_hypothesis.txt --references=/content/drive/MyDrive/summarization/evaluation/lstm_reference.txt --no-skipthoughts --no-glove 